# Step 5: CNN (Convolutional Neural Network) - 이미지 인식의 핵심

CNN은 이미지 처리에 특화된 신경망입니다. 이미지의 공간적 구조를 활용하여 효율적으로 특징을 추출합니다.

## 학습 목표
1. CNN의 핵심 구성 요소 이해 (Convolution, Pooling)
2. 이미지 데이터 전처리와 증강
3. CNN 아키텍처 설계
4. MNIST 손글씨 숫자 분류
5. CIFAR-10 컬러 이미지 분류

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

## 1. CNN의 핵심 개념

### 1.1 Convolution 연산 이해하기

In [ ]:
# 간단한 2D Convolution 예제
# 입력 이미지 (1채널, 5x5)
input_image = torch.tensor([
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 2, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
], dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # (1, 1, 5, 5)

# 3x3 커널 (엣지 검출)
kernel = torch.tensor([
    [-1, -1, -1],
    [-1,  8, -1],
    [-1, -1, -1]
], dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # (1, 1, 3, 3)

# Convolution 연산
conv = nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=False)
conv.weight.data = kernel

output = conv(input_image)

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# 입력 이미지
axes[0].imshow(input_image.squeeze(), cmap='gray')
axes[0].set_title('Input Image')
axes[0].grid(True, alpha=0.3)

# 커널
axes[1].imshow(kernel.squeeze(), cmap='RdBu')
axes[1].set_title('Kernel (Edge Detection)')
axes[1].grid(True, alpha=0.3)

# 출력
axes[2].imshow(output.squeeze().detach(), cmap='gray')
axes[2].set_title('Output (Convolved)')
axes[2].grid(True, alpha=0.3)

for ax in axes:
    ax.set_xticks(range(5))
    ax.set_yticks(range(5))

plt.tight_layout()
plt.show()

### 1.2 Pooling 연산

In [ ]:
# Pooling 예제
# 4x4 입력
input_pool = torch.tensor([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
], dtype=torch.float32).unsqueeze(0).unsqueeze(0)

# Max Pooling과 Average Pooling
max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)

max_output = max_pool(input_pool)
avg_output = avg_pool(input_pool)

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# 원본
im1 = axes[0].imshow(input_pool.squeeze(), cmap='viridis')
axes[0].set_title('Original (4x4)')
for i in range(4):
    for j in range(4):
        axes[0].text(j, i, f'{int(input_pool[0, 0, i, j])}', 
                    ha='center', va='center', color='white')
plt.colorbar(im1, ax=axes[0])

# Max Pooling
im2 = axes[1].imshow(max_output.squeeze(), cmap='viridis')
axes[1].set_title('Max Pooling (2x2)')
for i in range(2):
    for j in range(2):
        axes[1].text(j, i, f'{int(max_output[0, 0, i, j])}', 
                    ha='center', va='center', color='white')
plt.colorbar(im2, ax=axes[1])

# Average Pooling
im3 = axes[2].imshow(avg_output.squeeze(), cmap='viridis')
axes[2].set_title('Average Pooling (2x2)')
for i in range(2):
    for j in range(2):
        axes[2].text(j, i, f'{avg_output[0, 0, i, j]:.1f}', 
                    ha='center', va='center', color='white')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

## 2. MNIST 데이터셋으로 시작하기

MNIST는 0-9까지의 손글씨 숫자 이미지 데이터셋입니다.

In [ ]:
# 데이터 변환 정의
transform = transforms.Compose([
    transforms.ToTensor(),  # PIL Image → Tensor
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 평균과 표준편차로 정규화
])

# MNIST 데이터셋 다운로드
train_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=False, 
    download=True, 
    transform=transform
)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"학습 데이터: {len(train_dataset)} 샘플")
print(f"테스트 데이터: {len(test_dataset)} 샘플")

# 샘플 이미지 확인
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.ravel()

for idx in range(10):
    image, label = train_dataset[idx]
    axes[idx].imshow(image.squeeze(), cmap='gray')
    axes[idx].set_title(f'Label: {label}')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 3. 첫 번째 CNN 모델: LeNet-5 스타일

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)  # 28x28x1 → 28x28x6
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)       # 28x28x6 → 14x14x6
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)            # 14x14x6 → 10x10x16
        # 10x10x16 → 5x5x16 (after pooling)
        
        # Fully connected layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 10 classes
        
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        # Conv block 1
        x = self.pool(F.relu(self.conv1(x)))
        # Conv block 2
        x = self.pool(F.relu(self.conv2(x)))
        
        # Flatten
        x = x.view(-1, 16 * 5 * 5)
        
        # FC layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x

# 모델 생성
model = SimpleCNN().to(device)
print("모델 구조:")
print(model)

# 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n총 파라미터 수: {total_params:,}")
print(f"학습 가능한 파라미터 수: {trainable_params:,}")

### 3.1 모델 학습

In [ ]:
def train_epoch(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch}')
    for batch_idx, (data, target) in enumerate(progress_bar):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        progress_bar.set_postfix({'loss': loss.item()})
    
    train_loss /= len(train_loader)
    train_acc = correct / len(train_loader.dataset)
    
    return train_loss, train_acc

def test_epoch(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader)
    test_acc = correct / len(test_loader.dataset)
    
    return test_loss, test_acc

# 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# 학습
epochs = 10
train_losses, test_losses = [], []
train_accs, test_accs = [], []

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train_epoch(model, device, train_loader, optimizer, criterion, epoch)
    test_loss, test_acc = test_epoch(model, device, test_loader, criterion)
    scheduler.step()
    
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    
    print(f'\nEpoch: {epoch}')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

In [ ]:
# 학습 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 손실
axes[0].plot(train_losses, label='Train Loss')
axes[0].plot(test_losses, label='Test Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Test Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 정확도
axes[1].plot(train_accs, label='Train Accuracy')
axes[1].plot(test_accs, label='Test Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training and Test Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.2 예측 결과 확인

In [ ]:
# 예측 시각화
model.eval()
fig, axes = plt.subplots(3, 5, figsize=(12, 8))
axes = axes.ravel()

# 테스트 데이터에서 샘플 가져오기
test_iter = iter(test_loader)
images, labels = next(test_iter)

with torch.no_grad():
    outputs = model(images.to(device))
    _, predicted = torch.max(outputs, 1)

for idx in range(15):
    image = images[idx].squeeze()
    true_label = labels[idx].item()
    pred_label = predicted[idx].item()
    
    axes[idx].imshow(image, cmap='gray')
    axes[idx].set_title(f'True: {true_label}, Pred: {pred_label}')
    axes[idx].axis('off')
    
    # 틀린 예측은 빨간색으로 표시
    if true_label != pred_label:
        axes[idx].set_title(f'True: {true_label}, Pred: {pred_label}', color='red')

plt.tight_layout()
plt.show()

## 4. 더 깊은 CNN: 현대적인 아키텍처

In [ ]:
class ModernCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ModernCNN, self).__init__()
        
        # Convolutional blocks with batch normalization
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )
        
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )
        
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )
        
        # Global Average Pooling
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        
        x = self.classifier(x)
        return x

# 모델 생성
modern_model = ModernCNN().to(device)
print("현대적인 CNN 구조:")
print(modern_model)

# 파라미터 수
total_params = sum(p.numel() for p in modern_model.parameters())
print(f"\n총 파라미터 수: {total_params:,}")

## 5. CIFAR-10: 컬러 이미지 분류

이제 더 복잡한 32x32 컬러 이미지를 분류해봅시다.

In [ ]:
# CIFAR-10 데이터 전처리
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# CIFAR-10 데이터셋
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train
)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test
)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# 클래스 이름
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

# 샘플 이미지 시각화
def imshow(img):
    img = img / 2 + 0.5  # 정규화 해제
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

# 배치 가져오기
dataiter = iter(trainloader)
images, labels = next(dataiter)

# 이미지 보여주기
fig, axes = plt.subplots(2, 8, figsize=(16, 4))
axes = axes.ravel()

for idx in range(16):
    img = images[idx]
    img = img / 2 + 0.5  # 정규화 해제
    axes[idx].imshow(np.transpose(img, (1, 2, 0)))
    axes[idx].set_title(classes[labels[idx]])
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 5.1 CIFAR-10을 위한 CNN

In [ ]:
class CIFAR10CNN(nn.Module):
    def __init__(self):
        super(CIFAR10CNN, self).__init__()
        
        # VGG 스타일 블록
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.classifier = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# 모델 생성
cifar_model = CIFAR10CNN().to(device)
print(f"CIFAR-10 CNN 파라미터 수: {sum(p.numel() for p in cifar_model.parameters()):,}")

## 6. 특징 시각화

CNN이 학습한 특징들을 시각화해봅시다.

In [ ]:
def visualize_filters(model):
    # 첫 번째 컨볼루션 레이어의 필터 가져오기
    first_conv_layer = None
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            first_conv_layer = module
            break
    
    if first_conv_layer is None:
        print("Conv2d 레이어를 찾을 수 없습니다.")
        return
    
    # 필터 가중치 가져오기
    filters = first_conv_layer.weight.data.cpu()
    n_filters = min(filters.shape[0], 32)  # 최대 32개 필터만 표시
    
    # 시각화
    fig, axes = plt.subplots(4, 8, figsize=(16, 8))
    axes = axes.ravel()
    
    for i in range(n_filters):
        # 필터 정규화
        filter_img = filters[i]
        
        # 단일 채널인 경우 (MNIST)
        if filter_img.shape[0] == 1:
            filter_img = filter_img.squeeze()
            axes[i].imshow(filter_img, cmap='gray')
        # 3채널인 경우 (CIFAR-10)
        else:
            filter_img = filter_img.permute(1, 2, 0)
            # 정규화
            filter_img = (filter_img - filter_img.min()) / (filter_img.max() - filter_img.min())
            axes[i].imshow(filter_img)
        
        axes[i].axis('off')
        axes[i].set_title(f'Filter {i}')
    
    # 남은 subplot 숨기기
    for i in range(n_filters, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# MNIST 모델의 필터 시각화
print("MNIST CNN의 첫 번째 레이어 필터:")
visualize_filters(model)

### 6.1 중간 레이어 활성화 시각화

In [ ]:
def visualize_activations(model, input_image, target_layers=None):
    model.eval()
    activations = []
    
    # Hook 함수 정의
    def hook_fn(module, input, output):
        activations.append(output)
    
    # Hook 등록
    hooks = []
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            hook = module.register_forward_hook(hook_fn)
            hooks.append(hook)
    
    # Forward pass
    with torch.no_grad():
        _ = model(input_image.unsqueeze(0).to(device))
    
    # Hook 제거
    for hook in hooks:
        hook.remove()
    
    # 활성화 시각화
    fig, axes = plt.subplots(len(activations), 8, figsize=(16, 2*len(activations)))
    if len(activations) == 1:
        axes = axes.reshape(1, -1)
    
    for layer_idx, activation in enumerate(activations):
        act = activation[0].cpu()  # 첫 번째 샘플
        n_channels = min(act.shape[0], 8)  # 최대 8개 채널만 표시
        
        for ch_idx in range(n_channels):
            axes[layer_idx, ch_idx].imshow(act[ch_idx], cmap='viridis')
            axes[layer_idx, ch_idx].axis('off')
            if ch_idx == 0:
                axes[layer_idx, ch_idx].set_ylabel(f'Conv {layer_idx+1}', rotation=0, labelpad=40)
        
        # 남은 subplot 숨기기
        for ch_idx in range(n_channels, 8):
            axes[layer_idx, ch_idx].axis('off')
    
    plt.suptitle('Layer Activations', fontsize=16)
    plt.tight_layout()
    plt.show()

# 테스트 이미지로 활성화 시각화
test_image, test_label = test_dataset[0]
print(f"테스트 이미지 레이블: {test_label}")

# 원본 이미지 표시
plt.figure(figsize=(3, 3))
plt.imshow(test_image.squeeze(), cmap='gray')
plt.title(f'Input Image (Label: {test_label})')
plt.axis('off')
plt.show()

# 활성화 시각화
visualize_activations(model, test_image)

## 7. 전이 학습 (Transfer Learning)

사전 학습된 모델을 활용하여 적은 데이터로도 좋은 성능을 얻을 수 있습니다.

In [ ]:
# 사전 학습된 ResNet 모델 불러오기
import torchvision.models as models

# ResNet18 모델 (ImageNet으로 사전 학습됨)
resnet = models.resnet18(pretrained=True)

# 모델 구조 확인
print("ResNet18 구조 (마지막 부분):")
print(list(resnet.children())[-2:])

# CIFAR-10을 위해 마지막 층 수정
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, 10)  # 10개 클래스

# 특징 추출기로 사용 (conv 레이어는 고정)
for param in resnet.parameters():
    param.requires_grad = False

# 마지막 층만 학습 가능하게
for param in resnet.fc.parameters():
    param.requires_grad = True

print(f"\n학습 가능한 파라미터 수: {sum(p.numel() for p in resnet.parameters() if p.requires_grad):,}")
print(f"전체 파라미터 수: {sum(p.numel() for p in resnet.parameters()):,}")

## 8. 데이터 증강 (Data Augmentation) 효과

In [ ]:
# 다양한 데이터 증강 기법
augmentation_transforms = [
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
]

# 원본 이미지
original_transform = transforms.Compose([transforms.ToTensor()])
original_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=original_transform)
original_image, label = original_dataset[0]

# 증강된 이미지들 생성
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

# 원본 이미지
axes[0].imshow(np.transpose(original_image.numpy(), (1, 2, 0)))
axes[0].set_title('Original')
axes[0].axis('off')

# 증강된 이미지들
for i, aug_transform in enumerate(augmentation_transforms, 1):
    augmented_transform = transforms.Compose([
        transforms.ToPILImage(),
        aug_transform,
        transforms.ToTensor()
    ])
    
    # 같은 증강을 두 번 적용
    for j in range(2):
        idx = i + j * 5
        if idx < len(axes):
            augmented = augmented_transform(original_image)
            axes[idx].imshow(np.transpose(augmented.numpy(), (1, 2, 0)))
            axes[idx].set_title(aug_transform.__class__.__name__)
            axes[idx].axis('off')

plt.suptitle(f'Data Augmentation Examples (Class: {classes[label]})', fontsize=16)
plt.tight_layout()
plt.show()

## 9. 연습 문제

In [ ]:
# 문제 1: Residual Block 구현
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        # 힌트: 
        # 1. 두 개의 3x3 conv 레이어
        # 2. BatchNorm과 ReLU
        # 3. skip connection을 위한 shortcut
        pass
    
    def forward(self, x):
        # 힌트: out = F.relu(residual + shortcut)
        pass

# 문제 2: Inception Module 구현
class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_1x1, out_3x3, out_5x5, out_pool):
        super(InceptionModule, self).__init__()
        # 힌트: 서로 다른 크기의 conv를 병렬로 수행하고 concatenate
        pass

# 문제 3: Attention 메커니즘 추가
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16):
        super(ChannelAttention, self).__init__()
        # 힌트: Global Average Pooling → FC → ReLU → FC → Sigmoid
        pass

## 정리

이번 튜토리얼에서 배운 내용:
1. CNN의 핵심 구성 요소 (Convolution, Pooling)
2. MNIST 손글씨 숫자 분류
3. 현대적인 CNN 아키텍처 설계
4. CIFAR-10 컬러 이미지 분류
5. 특징 시각화와 해석
6. 전이 학습
7. 데이터 증강

### CNN의 핵심 개념:
- **지역적 연결성**: 각 뉴런이 입력의 일부분만 봄
- **가중치 공유**: 같은 필터를 이미지 전체에 적용
- **계층적 특징 학습**: 낮은 층은 엣지, 높은 층은 복잡한 패턴
- **이동 불변성**: 객체가 이미지의 어디에 있어도 인식

다음 단계에서는 RNN을 사용하여 순차 데이터를 처리하는 방법을 배워보겠습니다!